In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import reciprocal, expon
from sklearn.model_selection import StratifiedShuffleSplit # Required for stratified split

# --- 1. Data Loading and Splitting (Self-contained for this notebook) ---
print("--- 1. Data Loading and Splitting ---")

def load_housing_data_for_models(housing_path="housing.csv"):
    try:
        housing = pd.read_csv(housing_path)
        print(f"Housing dataset loaded successfully from '{housing_path}'.")
        housing.columns = housing.columns.str.strip().str.lower().str.replace(' ', '_')
    except FileNotFoundError:
        print(f"Error: '{housing_path}' not found. Creating a synthetic dataset for demonstration.")
        np.random.seed(42) # for reproducibility
        n_samples = 2000
        data = {
            'longitude': np.random.uniform(-125.0, -114.0, n_samples),
            'latitude': np.random.uniform(32.0, 42.0, n_samples),
            'housing_median_age': np.random.randint(1, 55, n_samples),
            'total_rooms': np.random.randint(6, 15000, n_samples),
            'total_bedrooms': np.random.randint(1, 3000, n_samples),
            'population': np.random.randint(3, 10000, n_samples),
            'households': np.random.randint(1, 2500, n_samples),
            'median_income': np.random.uniform(0.5, 10, n_samples),
            'ocean_proximity': np.random.choice(['<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND'], n_samples, p=[0.4, 0.3, 0.15, 0.1, 0.05]),
            'median_house_value': np.random.uniform(10000, 600000, n_samples)
        }
        housing = pd.DataFrame(data)
        for col in ['total_bedrooms', 'median_income']:
            missing_indices = np.random.choice(housing.index, size=int(0.02 * n_samples), replace=False)
            housing.loc[missing_indices, col] = np.nan

    # Create income categories for stratified sampling
    housing["median_income"] = pd.to_numeric(housing["median_income"], errors='coerce')
    housing['median_income'].fillna(housing['median_income'].median(), inplace=True) # Fill NaN for income_cat creation
    housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
    housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

    # Perform stratified sampling
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(housing, housing["income_cat"]):
        strat_train_set = housing.loc[train_index]
        strat_test_set = housing.loc[test_index]

    # Drop the income_cat column
    for set_ in (strat_train_set, strat_test_set):
        set_.drop("income_cat", axis=1, inplace=True)

    X_train = strat_train_set.drop("median_house_value", axis=1)
    y_train = strat_train_set["median_house_value"].copy()
    X_test = strat_test_set.drop("median_house_value", axis=1)
    y_test = strat_test_set["median_house_value"].copy()

    return X_train, y_train, X_test, y_test

# Load data
X_train, y_train, X_test, y_test = load_housing_data_for_models()
print(f"Data loaded and split into X_train ({X_train.shape}), y_train ({y_train.shape}), X_test ({X_test.shape}), y_test ({y_test.shape}).")

# --- Preprocessing Pipeline Setup ---
# Dynamically identify numerical and categorical features
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include='object').columns.tolist()

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Impute before OneHotEncoder
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Keep any other columns not explicitly transformed
)
print("Preprocessing pipelines defined for numerical and categorical features.")

# --- 2. Train and Evaluate Each Model ---
print("\n--- 2. Training and Evaluating Models ---")
results = [] # List to store model results

# --- Linear Regression ---
print("\n--- Training Linear Regression ---")
pipeline_lr = Pipeline([('preprocessor', preprocessor), ('lin_reg', LinearRegression())])
pipeline_lr.fit(X_train, y_train)
lin_reg_predictions = pipeline_lr.predict(X_test)
lin_reg_rmse = np.sqrt(mean_squared_error(y_test, lin_reg_predictions))
lin_reg_scores = cross_val_score(pipeline_lr, X_train, y_train, scoring="neg_mean_squared_error", cv=3, n_jobs=-1)
lin_reg_rmse_scores = np.sqrt(-lin_reg_scores)
print(f"Linear Regression Test RMSE: {lin_reg_rmse:.2f}")
print(f"Linear Regression CV RMSE (Mean): {np.mean(lin_reg_rmse_scores):.2f}")
results.append({
    "Model/Configuration": "Linear Regression",
    "Best Hyperparameters": "N/A",
    "CV RMSE (Mean)": np.mean(lin_reg_rmse_scores),
    "Test RMSE": lin_reg_rmse,
    "Notes": "Baseline model"
})

# --- Decision Tree Regressor ---
print("\n--- Training Decision Tree Regressor ---")
pipeline_tree = Pipeline([('preprocessor', preprocessor), ('tree_reg', DecisionTreeRegressor(random_state=42))])
pipeline_tree.fit(X_train, y_train)
tree_predictions = pipeline_tree.predict(X_test)
tree_rmse = np.sqrt(mean_squared_error(y_test, tree_predictions))
tree_reg_scores = cross_val_score(pipeline_tree, X_train, y_train, scoring="neg_mean_squared_error", cv=3, n_jobs=-1)
tree_reg_rmse_scores = np.sqrt(-tree_reg_scores)
print(f"Decision Tree Regressor Test RMSE: {tree_rmse:.2f}")
print(f"Decision Tree Regressor CV RMSE (Mean): {np.mean(tree_reg_rmse_scores):.2f}")
results.append({
    "Model/Configuration": "Decision Tree Regressor",
    "Best Hyperparameters": "N/A",
    "CV RMSE (Mean)": np.mean(tree_reg_rmse_scores),
    "Test RMSE": tree_rmse,
    "Notes": "Single tree, prone to overfitting"
})

# --- Random Forest Regressor ---
print("\n--- Training Random Forest Regressor ---")
pipeline_forest = Pipeline([('preprocessor', preprocessor), ('forest_reg', RandomForestRegressor(random_state=42))])
pipeline_forest.fit(X_train, y_train)
forest_predictions = pipeline_forest.predict(X_test)
forest_rmse = np.sqrt(mean_squared_error(y_test, forest_predictions))
forest_reg_scores = cross_val_score(pipeline_forest, X_train, y_train, scoring="neg_mean_squared_error", cv=3, n_jobs=-1)
forest_reg_rmse_scores = np.sqrt(-forest_reg_scores)
print(f"Random Forest Regressor Test RMSE: {forest_rmse:.2f}")
print(f"Random Forest Regressor CV RMSE (Mean): {np.mean(forest_reg_rmse_scores):.2f}")
results.append({
    "Model/Configuration": "Random Forest Regressor",
    "Best Hyperparameters": "N/A",
    "CV RMSE (Mean)": np.mean(forest_reg_rmse_scores),
    "Test RMSE": forest_rmse,
    "Notes": "Ensemble of decision trees, generally robust"
})

# --- SVR with GridSearchCV ---
print("\n--- Training SVR with GridSearchCV ---")
pipeline_svr = Pipeline([('preprocessor', preprocessor), ('svr', SVR())])
param_grid_svr = [
    {'svr__kernel': ['linear'], 'svr__C': [0.1, 1.0]},
    {'svr__kernel': ['rbf'], 'svr__C': [0.1, 1.0], 'svr__gamma': [0.01, 0.1]}
]
# Reduce cv for faster execution for demonstration, increase for better search
grid_search_svr = GridSearchCV(pipeline_svr, param_grid_svr, cv=2,
                               scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_svr.fit(X_train, y_train)
best_svr_grid_model = grid_search_svr.best_estimator_
test_predictions_svr_grid = best_svr_grid_model.predict(X_test)
test_rmse_svr_grid = np.sqrt(mean_squared_error(y_test, test_predictions_svr_grid))
print(f"SVR (GridSearchCV) Best Parameters: {grid_search_svr.best_params_}")
print(f"SVR (GridSearchCV) CV RMSE (Best Score): {np.sqrt(-grid_search_svr.best_score_):.2f}")
print(f"SVR (GridSearchCV) Test RMSE: {test_rmse_svr_grid:.2f}")
results.append({
    "Model/Configuration": "SVR (GridSearchCV)",
    "Best Hyperparameters": grid_search_svr.best_params_,
    "CV RMSE (Mean)": np.sqrt(-grid_search_svr.best_score_),
    "Test RMSE": test_rmse_svr_grid,
    "Notes": "Exhaustive search for SVR hyperparameters"
})


# --- SVR with RandomizedSearchCV ---
print("\n--- Training SVR with RandomizedSearchCV ---")
pipeline_svr_rand = Pipeline([('preprocessor', preprocessor), ('svr', SVR())])
param_distributions_svr_rand = {
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': reciprocal(0.1, 100),
    'svr__gamma': expon(scale=0.1)
}
# Reduce n_iter and cv for faster execution for demonstration
random_search_svr = RandomizedSearchCV(pipeline_svr_rand, param_distributions_svr_rand,
                                       n_iter=5, cv=2,
                                       scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, verbose=0)
random_search_svr.fit(X_train, y_train)
best_svr_rand_model = random_search_svr.best_estimator_
test_predictions_svr_rand = best_svr_rand_model.predict(X_test)
test_rmse_svr_rand = np.sqrt(mean_squared_error(y_test, test_predictions_svr_rand))
print(f"SVR (RandomizedSearchCV) Best Parameters: {random_search_svr.best_params_}")
print(f"SVR (RandomizedSearchCV) CV RMSE (Best Score): {np.sqrt(-random_search_svr.best_score_):.2f}")
print(f"SVR (RandomizedSearchCV) Test RMSE: {test_rmse_svr_rand:.2f}")
results.append({
    "Model/Configuration": "SVR (RandomizedSearchCV)",
    "Best Hyperparameters": random_search_svr.best_params_,
    "CV RMSE (Mean)": np.sqrt(-random_search_svr.best_score_),
    "Test RMSE": test_rmse_svr_rand,
    "Notes": "Random sampling for SVR hyperparameters, more efficient"
})


# --- Saving the best model as 'newmodel.pkl' ---
# The model with the lowest CV RMSE (best_score_) from GridSearchCV is chosen as "newmodel.pkl"
# You might choose a different model based on your comparison results.
newmodel = best_svr_grid_model # Assuming GridSearchCV SVR is the best based on typical results
joblib.dump(newmodel, "newmodel.pkl")
print("\nFinal trained model saved as 'newmodel.pkl'. This model includes preprocessing steps.")

# --- 3. Create the Model Comparison Table ---
print("\n--- 3. Final Model Comparison Table ---")
comparison_df = pd.DataFrame(results)
print(comparison_df.to_string())

# You can optionally save the table
comparison_df.to_csv("model_comparison_table.csv", index=False)
print("\nModel comparison table saved as 'model_comparison_table.csv'.")

--- 1. Data Loading and Splitting ---
Housing dataset loaded successfully from 'housing.csv'.
Data loaded and split into X_train ((16512, 9)), y_train ((16512,)), X_test ((4128, 9)), y_test ((4128,)).
Preprocessing pipelines defined for numerical and categorical features.

--- 2. Training and Evaluating Models ---

--- Training Linear Regression ---
Linear Regression Test RMSE: 67346.88
Linear Regression CV RMSE (Mean): 69292.91

--- Training Decision Tree Regressor ---
Decision Tree Regressor Test RMSE: 69203.53
Decision Tree Regressor CV RMSE (Mean): 69846.11

--- Training Random Forest Regressor ---
Random Forest Regressor Test RMSE: 47197.67
Random Forest Regressor CV RMSE (Mean): 50498.68

--- Training SVR with GridSearchCV ---
SVR (GridSearchCV) Best Parameters: {'svr__C': 1.0, 'svr__kernel': 'linear'}
SVR (GridSearchCV) CV RMSE (Best Score): 115247.64
SVR (GridSearchCV) Test RMSE: 110171.50

--- Training SVR with RandomizedSearchCV ---
SVR (RandomizedSearchCV) Best Parameters: {